# Dependency

In [ ]:
!pip install flask --quiet
!pip install flask-ngrok --quiet
!pip install pyngrok --quiet
!pip install openai --quiet
!pip install langchain --quiet
!pip install sentence_transformers --quiet
!pip install faiss-cpu --quiet
!pip install tiktoken --quiet

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz --quiet
!tar -xvf /content/ngrok-stable-linux-amd64.tgz --quiet

# Mount at GoogleDrive

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/jinwoo/
ROOT_PATH = "/content/drive/MyDrive/jinwoo/"

Mounted at /content/drive
/content/drive/MyDrive/jinwoo


# openAI API Key

In [8]:
import os
import openai

with open(ROOT_PATH+'keys.txt', 'r') as f:
    OPEN_API_KEY = f.readline()
    # NGROK_AUTH_KEY = f.readline()

openai.api_key = OPEN_API_KEY
os.environ["OPENAI_API_KEY"] = OPEN_API_KEY

In [9]:
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS, Chroma
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader, DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

# LLM chat & Vector Index

In [12]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)
embeddings = OpenAIEmbeddings()

fdb = FAISS.load_local(folder_path = ROOT_PATH+"faiss-nj",
                       embeddings = embeddings)

index = VectorStoreIndexWrapper(vectorstore = fdb)

# Flask App

In [13]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import requests
import json

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/chatbot', methods = ['POST'])
def chatbot():
    request_message = request.get_json()
    response = index.query(request_message["message"], llm = chat, verbose = True)
    return jsonify({"response": response})

In [14]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://646f-34-73-75-241.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
